In [1]:
# Importando API, Libs e Funções helper

# API
from riotwatcher import LolWatcher, ApiError

# Helpers
from roleidentification import pull_data, get_roles
from helper import players_perfomance_at_n, events_at_n, get_participant_game_info, create_match_row
from helper import create_matchlist_from_summoner, get_top_players_users

# Libs de Manipulação dos Dados
import numpy as np
import pandas as pd

# Jovian
import jovian

In [ ]:
import jovian
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..


In [ ]:
# Inicialmente, vou criar um DataFrame contendo as colunas de interesse

all_matches = pd.DataFrame(columns = ['gameID', 'isWinner_blue', 'totalGold_red', 'xp_red', 'nKills_red', 'nDeaths_red',
                                      'nAssists_red', 'minionsKilled_red', 'jungleMinionsKilled_red', 'wardsPlaced_red',
                                      'wardsKilled_red','firstBlood_red', 'firstTower_red', 'midTowersDestroyed_red',
                                      'botTowersDestroyed_red', 'topTowersDestroyed_red', 'inhibitorsDestroyed_red',
                                      'fireDragonsDestroyed_red','airDragonsDestroyed_red','waterDragonsDestroyed_red',
                                      'earthDragonsDestroyed_red', 'riftHeraldDestroyed_red', 'TOP_red', 'JUNGLE_red',
                                      'MIDDLE_red', 'BOTTOM_red', 'UTILITY_red','totalGold_blue', 'xp_blue', 'nKills_blue',
                                      'nDeaths_blue', 'nAssists_blue', 'minionsKilled_blue', 'jungleMinionsKilled_blue',
                                      'wardsPlaced_blue', 'wardsKilled_blue','firstBlood_blue', 'firstTower_blue', 
                                      'midTowersDestroyed_blue', 'botTowersDestroyed_blue', 'topTowersDestroyed_blue',
                                      'inhibitorsDestroyed_blue', 'fireDragonsDestroyed_blue','airDragonsDestroyed_blue',
                                      'waterDragonsDestroyed_blue','earthDragonsDestroyed_blue','riftHeraldDestroyed_blue',
                                      'TOP_blue', 'JUNGLE_blue', 'MIDDLE_blue', 'BOTTOM_blue', 'UTILITY_blue'])

# Listas para armazenar players e matchs já vistas
users_todos = []
match_id_list = []

In [ ]:
# Variáveis Globais
api_key = 'RGAPI-xxx'
region = 'br1'

# Instanciar a função que facilita na identificação das roles de acordo com a ID de cada champion
champion_roles = pull_data()

# Intanciar a API do LOL - MatchV4 e LeagueV4
lol_watcher = LolWatcher(api_key)

m = lol_watcher.match
leagues = lol_watcher.league

# Função helper que retorna nome dos players a partir do diamante 1
top_players_br = get_top_players_users(leagues, region = region)

Devido a forma que a API do lol funciona e o tempo que toma para realizar esse processo, a query especifica que eu quero
fazer (centenas de jogadores diferentes e controlando o begin_time) vai gerar **HTTPError** diversas vezes, em diferentes
situações. 
<br>

Para resolver problema, normalmente bastava eu **adicionar o nome do player que estava causando o problema aos users já vistos**, mas nem sempre o nome do player estava na mensagem de erro (o que impossibilitou pegar o erro e automatizar 100% a solução). Além disso, como a ideia era utilizar um dataset grande (50k >) e há um limite de requests na API, tive que realizar isso manualmente também.  
<br>

O processo de obtenção das 50mil partidas tomou um ou 2 dias, sem contar com a definição das funções helpers.

In [ ]:
for user in top_players_br:
    
    if user not in users_todos:
    
        # Instanciar API por player - reduziu o número de vezes que a solicitação deu erro
        lol_watcher = LolWatcher(api_key)
        m = lol_watcher.match

        # Buscar dados de summoner do player e procurar as partidas desde 12/Fev  
        summoner_data = lol_watcher.summoner.by_name(region, user)

        match_list = m.matchlist_by_account(**{'region': region,
                                              'encrypted_account_id': summoner_data['accountId'],
                                              'begin_time': 1613171498})
        users_todos.append(user)

        n_matches = len(match_list['matches'])

        for n_match in range(n_matches):

            match_id = match_list['matches'][n_match]['gameId']

            # id 420 - Partidas SoloQ Ranked
            if match_list['matches'][n_match]['queue'] == 420:

                # Como os players jogam um contra o outro e são do mesmo nível, procuro ver se a partida já não existe na base
                if match_id not in match_id_list:

                    match_id_list.append(match_id)

                    try:
                        game_info = m.by_id('br1', match_id)
                        timeline = m.timeline_by_match(region = region, match_id = match_id)

                        # Apenas partidas maiores que 10min
                        if len(timeline['frames']) > 10:

                            players_perfomance_at_10 = players_perfomance_at_n(timeline)
                            players_events_at_10 = events_at_n(timeline)
                            participant_game_info = get_participant_game_info(game_info)

                            match_row = create_match_row(players_perfomance_at_10, players_events_at_10, participant_game_info, champion_roles, match_id)

                            all_matches = all_matches.append(match_row)

                    # Except do HTTP Error (que não funcionou mt bem hehe)
                    except ApiError:
                        
                        # Instanciar API por erro também reduziu o número de vezes que a solicitação deu erro (MUITO)
                        lol_watcher = LolWatcher(api_key)
                        m = lol_watcher.match

In [ ]:
# all_matches.to_csv('54k_matches___.csv')